In [3]:
import os
import requests
import json
import pandas as pd
import concurrent.futures
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# Set your API keys here
FALLBACK_PERPLEXITY_API_KEY = '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'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PERPLEXITY_API_KEY = os.getenv('PERPLEXITY_API_KEY', FALLBACK_PERPLEXITY_API_KEY)

# Check if the API keys are set
if not OPENAI_API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")
if not PERPLEXITY_API_KEY:
    raise ValueError("Please set the PERPLEXITY_API_KEY environment variable or provide a fallback API key.")

# Create a requests session with retries
session = requests.Session()
retry = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# Initialize the ChatOpenAI model
chat = ChatOpenAI(temperature=0.7, openai_api_key=OPENAI_API_KEY)

# 📊 Function to gather latest real estate news
def get_real_estate_news():
    query = "latest real estate news in North Texas"
    url = f"https://api.perplexity.ai/chat/completions"
    headers = {
        "Authorization": f"Bearer {PERPLEXITY_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "llama-3-sonar-small-32k-online",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": query}
        ],
        "max_tokens": 4000,
        "temperature": 0.7,
        "top_p": 0.95,
        "return_citations": True
    }
    response = session.post(url, headers=headers, data=json.dumps(payload))
    response.raise_for_status()
    print("Successfully retrieved news")
    return response.json()['choices'][0]['message']['content']

# 🖼️ Function to create an image for each news summary
def create_image(prompt):
    url = "https://api.openai.com/v1/images/generations"
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "dall-e-3",
        "prompt": prompt,
        "n": 1,
        "size": "1024x1024"
    }
    response = session.post(url, headers=headers, json=data)
    response.raise_for_status()
    print(f"Successfully created image for prompt: {prompt}")
    return response.json()['data'][0]['url']

# 🖥️ Function to generate HTML content
def generate_html(news_items):
    html_content = """
   <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Real Estate News in North Texas</title>
    <style>
        body {
            font-family: 'Helvetica Neue', Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #ffffff;
            color: #333333;
        }
        .container {
            max-width: 800px;
            margin: 40px auto;
            padding: 20px;
            background-color: #f7f7f7;
            border-radius: 10px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }
        header {
            text-align: center;
            padding: 20px;
        }
        header h1 {
            font-size: 2em;
            margin-bottom: 10px;
            color: #333333;
        }
        header p {
            font-size: 1em;
            margin-bottom: 10px;
            color: #666666;
        }
        .news-container {
            padding: 20px;
        }
        .news-item {
            margin-bottom: 40px;
        }
        .news-item img {
            width: 100%;
            height: auto;
            border-radius: 5px;
            margin-bottom: 15px;
        }
        .news-summary {
            font-size: 1em;
            line-height: 1.6;
            color: #666666;
        }
        .contact-info {
            text-align: center;
            margin-top: 20px;
            font-size: 1em;
            padding: 10px;
            background-color: #e0e0e0;
            border-radius: 5px;
        }
        .contact-info a {
            color: #007bff;
            text-decoration: none;
        }
        .contact-info a:hover {
            text-decoration: underline;
        }
        footer {
            text-align: center;
            padding: 20px;
            font-size: 0.9em;
            color: #999999;
        }
    </style>
</head>
<body>
    <div class="container">
        <header>
            <h1>Real Estate Insights by Michelle Sanchez</h1>
            <p>Welcome to today's update on the latest in real estate!</p>
            <p>Contact me anytime at <a href="mailto:Michelledsanchez413@gmail.com">Michelledsanchez413@gmail.com</a> or call me directly at 940-273-4848.</p>
            <p>Stay connected on <a href="https://www.facebook.com/MichelleSanchezRealtor">Facebook</a> for more updates and listings.</p>
        </header>
        <div class="news-container">
            <!-- Repeat this block for each news item -->
            <div class="news-item">
                <img src="image-url.jpg" alt="News Image">
                <div class="news-summary">
                    <!-- News summary content goes here -->
                </div>
            </div>
            <!-- End of news item block -->
        </div>
        <div class="contact-info">
            <p>Contact me anytime at <a href="mailto:Michelledsanchez413@gmail.com">Michelledsanchez413@gmail.com</a> or call me directly at 940-273-4848.</p>
            <p>Stay connected on <a href="https://www.facebook.com/MichelleSanchezRealtor">Facebook</a> for more updates and listings.</p>
        </div>
    </div>
    <footer>
        <p>&copy; 2024 Michelle Sanchez Real Estate. All rights reserved.</p>
    </footer>
</body>
</html>

    """
    for news in news_items:
        html_content += f"""
        <div class="news-container">
            <h2>{news['title']}</h2>
            <img src="{news['image_url']}" alt="Image for {news['title']}">
            <p class="news-summary">{news['summary']}</p>
            <p><a href="{news['original_link']}" target="_blank">Read more</a></p>
        </div>
        """
    html_content += """
            <div class="contact-info">
                <p>For more information, visit my <a href="https://www.facebook.com/MichelleSanchezRealtor">Facebook page</a> or contact me directly.</p>
            </div>
        </div>
    </body>
    </html>
    """
    return html_content

# Function to process news and create images concurrently
def process_news_and_create_images(news_content):
    news_items = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_image = {
            executor.submit(create_image, f"An illustration representing the following real estate news: {news.strip()}"): news
            for news in news_content.split("\n\n")
        }
        for future in concurrent.futures.as_completed(future_to_image):
            news = future_to_image[future]
            try:
                image_url = future.result()
                news_items.append({
                    "title": f"Real Estate News {len(news_items) + 1}",
                    "summary": news.strip(),
                    "image_url": image_url,
                    "original_link": "https://example.com"  # Replace with actual link if available
                })
            except Exception as e:
                print(f"Error creating image for news: {news.strip()}. Error: {e}")

    return news_items

# Main workflow function
def main():
    try:
        # Fetch news content
        news_content = get_real_estate_news()

        # Process news and create images concurrently
        news_items = process_news_and_create_images(news_content)

        # Generate HTML content
        html_content = generate_html(news_items)

        # Save HTML content
        with open('real_estate_news_north_texasv2.html', 'w') as f:
            f.write(html_content)
        print("HTML content has been saved to 'real_estate_news_north_texasv2.html'.")

        # Placeholder for Facebook posting
        print("Placeholder for Facebook posting")

    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()

Successfully retrieved news
Successfully created image for prompt: An illustration representing the following real estate news: 3. **Investment and Market Trends**:
   - Real estate investors are buying U.S. homes at the fastest pace in nearly two years, driven by market conditions and housing affordability.
   - An overwhelming majority of homes in the U.S. are overvalued due to steep mortgage rates and the ongoing housing shortage, pushing prices even higher.
Successfully created image for prompt: An illustration representing the following real estate news: 4. **Community Development and Social Impact**:
   - Entrepreneur Mikail Onu aims to improve neighborhoods in neglected communities in Dallas by creating more affordable and sustainable housing options.
   - A new study suggests that the location of a homebuyer's mom plays a significant role in purchasing decisions, highlighting the importance of family and community in real estate decisions.
Successfully created image for prompt:

In [4]:
import os
import requests
import json
import pandas as pd
import concurrent.futures
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# Set your API keys here
FALLBACK_PERPLEXITY_API_KEY = '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'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PERPLEXITY_API_KEY = os.getenv('PERPLEXITY_API_KEY', FALLBACK_PERPLEXITY_API_KEY)

# Check if the API keys are set
if not OPENAI_API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")
if not PERPLEXITY_API_KEY:
    raise ValueError("Please set the PERPLEXITY_API_KEY environment variable or provide a fallback API key.")

# Create a requests session with retries
session = requests.Session()
retry = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# Initialize the ChatOpenAI model
chat = ChatOpenAI(temperature=0.7, openai_api_key=OPENAI_API_KEY)

# 📊 Function to gather latest real estate news
def get_real_estate_news():
    query = "latest real estate news in North Texas"
    url = f"https://api.perplexity.ai/chat/completions"
    headers = {
        "Authorization": f"Bearer {PERPLEXITY_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "llama-3-sonar-small-32k-online",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": query}
        ],
        "max_tokens": 4000,
        "temperature": 0.7,
        "top_p": 0.95,
        "return_citations": True
    }
    response = session.post(url, headers=headers, data=json.dumps(payload))
    response.raise_for_status()
    print("Successfully retrieved news")
    return response.json()['choices'][0]['message']['content']

# 🖼️ Function to create an image for each news summary
def create_image(prompt):
    url = "https://api.openai.com/v1/images/generations"
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "dall-e-3",
        "prompt": prompt,
        "n": 1,
        "size": "1024x1024"
    }
    response = session.post(url, headers=headers, json=data)
    response.raise_for_status()
    print(f"Successfully created image for prompt: {prompt}")
    return response.json()['data'][0]['url']

# 🖥️ Function to generate HTML content
def generate_html(news_items):
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Real Estate News in North Texas</title>
        <style>
            body {
                font-family: 'Helvetica Neue', Arial, sans-serif;
                margin: 0;
                padding: 0;
                background-color: #ffffff;
                color: #333333;
            }
            .container {
                max-width: 800px;
                margin: 40px auto;
                padding: 20px;
                background-color: #f7f7f7;
                border-radius: 10px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }
            header {
                text-align: center;
                padding: 20px;
            }
            header h1 {
                font-size: 2em;
                margin-bottom: 10px;
                color: #333333;
            }
            header p {
                font-size: 1em;
                margin-bottom: 10px;
                color: #666666;
            }
            .news-container {
                padding: 20px;
            }
            .news-item {
                margin-bottom: 40px;
            }
            .news-item img {
                width: 100%;
                height: auto;
                border-radius: 5px;
                margin-bottom: 15px;
            }
            .news-summary {
                font-size: 1em;
                line-height: 1.6;
                color: #666666;
            }
            .contact-info {
                text-align: center;
                margin-top: 20px;
                font-size: 1em;
                padding: 10px;
                background-color: #e0e0e0;
                border-radius: 5px;
            }
            .contact-info a {
                color: #007bff;
                text-decoration: none;
            }
            .contact-info a:hover {
                text-decoration: underline;
            }
            footer {
                text-align: center;
                padding: 20px;
                font-size: 0.9em;
                color: #999999;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <header>
                <h1>Real Estate Insights by Michelle Sanchez</h1>
                <p>Welcome to today's update on the latest in real estate!</p>
                <p>Contact me anytime at <a href="mailto:Michelledsanchez413@gmail.com">Michelledsanchez413@gmail.com</a> or call me directly at 940-273-4848.</p>
                <p>Stay connected on <a href="https://www.facebook.com/MichelleSanchezRealtor">Facebook</a> for more updates and listings.</p>
            </header>
            <div class="news-container">
    """
    for news in news_items:
        html_content += f"""
            <div class="news-item">
                <h2>{news['title']}</h2>
                <img src="{news['image_url']}" alt="Image for {news['title']}">
                <p class="news-summary">{news['summary']}</p>
                <p><a href="{news['original_link']}" target="_blank">Read more</a></p>
            </div>
        """
    html_content += """
            </div>
            <div class="contact-info">
                <p>Contact me anytime at <a href="mailto:Michelledsanchez413@gmail.com">Michelledsanchez413@gmail.com</a> or call me directly at 940-273-4848.</p>
                <p>Stay connected on <a href="https://www.facebook.com/MichelleSanchezRealtor">Facebook</a> for more updates and listings.</p>
            </div>
        </div>
        <footer>
            <p>&copy; 2024 Michelle Sanchez Real Estate. All rights reserved.</p>
        </footer>
    </body>
    </html>
    """
    return html_content

# Function to process news and create images concurrently
def process_news_and_create_images(news_content):
    news_items = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_image = {
            executor.submit(create_image, f"An illustration representing the following real estate news: {news.strip()}"): news
            for news in news_content.split("\n\n")
        }
        for future in concurrent.futures.as_completed(future_to_image):
            news = future_to_image[future]
            try:
                image_url = future.result()
                news_items.append({
                    "title": f"Real Estate News {len(news_items) + 1}",
                    "summary": news.strip(),
                    "image_url": image_url,
                    "original_link": "https://example.com"  # Replace with actual link if available
                })
            except Exception as e:
                print(f"Error creating image for news: {news.strip()}. Error: {e}")

    return news_items

# Main workflow function
def main():
    try:
        # Fetch news content
        news_content = get_real_estate_news()

        # Process news and create images concurrently
        news_items = process_news_and_create_images(news_content)

        # Generate HTML content
        html_content = generate_html(news_items)

        # Save HTML content
        with open('real_estate_news_north_texasv4.html', 'w') as f:
            f.write(html_content)
        print("HTML content has been saved to 'real_estate_news_north_texasv4.html'.")

        # Placeholder for Facebook posting
        print("Placeholder for Facebook posting")

    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()


Successfully retrieved news
Successfully created image for prompt: An illustration representing the following real estate news: These updates highlight the complex dynamics in North Texas’s real estate market, particularly with regards to affordability, zoning, and commercial developments.
Successfully created image for prompt: An illustration representing the following real estate news: 1. **Home Prices and Affordability**: The US housing shortage has grown to 4.5 million homes amid an affordability crisis, with some cities ranked among the world’s most unaffordable places for housing. Rising home insurance premiums and steep mortgage rates continue to squeeze homeownership, leading to more people considering going back to renting.
Successfully created image for prompt: An illustration representing the following real estate news: 5. **Market Analysis**: The Dallas housing market trends indicate that overvalued homes are the norm, and the Fed’s handling of rate cuts will play a crucial

In [5]:
import os
import requests
import json
import pandas as pd
import concurrent.futures
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# Set your API keys here
FALLBACK_PERPLEXITY_API_KEY = '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'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PERPLEXITY_API_KEY = os.getenv('PERPLEXITY_API_KEY', FALLBACK_PERPLEXITY_API_KEY)

# Check if the API keys are set
if not OPENAI_API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")
if not PERPLEXITY_API_KEY:
    raise ValueError("Please set the PERPLEXITY_API_KEY environment variable or provide a fallback API key.")

# Create a requests session with retries
session = requests.Session()
retry = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# Initialize the ChatOpenAI model
chat = ChatOpenAI(temperature=0.7, openai_api_key=OPENAI_API_KEY)

# 📊 Function to gather latest real estate news
def get_real_estate_news():
    query = "latest real estate news in North Texas"
    url = f"https://api.perplexity.ai/chat/completions"
    headers = {
        "Authorization": f"Bearer {PERPLEXITY_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "llama-3-sonar-small-32k-online",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": query}
        ],
        "max_tokens": 4000,
        "temperature": 0.7,
        "top_p": 0.95,
        "return_citations": True
    }
    response = session.post(url, headers=headers, data=json.dumps(payload))
    response.raise_for_status()
    print("Successfully retrieved news")
    return response.json()['choices'][0]['message']['content']

# 🖼️ Function to create an image for each news summary
def create_image(prompt):
    url = "https://api.openai.com/v1/images/generations"
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "dall-e-3",
        "prompt": prompt,
        "n": 1,
        "size": "1024x1024"
    }
    response = session.post(url, headers=headers, json=data)
    response.raise_for_status()
    print(f"Successfully created image for prompt: {prompt}")
    return response.json()['data'][0]['url']

# 🖥️ Function to generate HTML content
def generate_html(news_items):
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Real Estate News in North Texas</title>
        <style>
            body {
                font-family: 'Georgia', serif;
                background-color: #f7f7f7;
                color: #333333;
                margin: 0;
                padding: 0;
            }
            .container {
                max-width: 800px;
                margin: 40px auto;
                padding: 20px;
                background-color: #ffffff;
                border-radius: 5px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }
            header {
                text-align: center;
                border-bottom: 2px solid #e0e0e0;
                padding-bottom: 20px;
                margin-bottom: 20px;
            }
            header h1 {
                font-size: 2.5em;
                margin: 0;
                font-weight: bold;
                color: #333333;
            }
            header p {
                font-size: 1.2em;
                color: #666666;
                margin: 10px 0 0;
            }
            .news-container {
                padding: 20px 0;
            }
            .news-item {
                border-bottom: 1px solid #e0e0e0;
                margin-bottom: 20px;
                padding-bottom: 20px;
            }
            .news-item:last-child {
                border-bottom: none;
                margin-bottom: 0;
                padding-bottom: 0;
            }
            .news-item img {
                width: 100%;
                height: auto;
                border-radius: 5px;
                margin-bottom: 10px;
            }
            .news-summary {
                font-size: 1.1em;
                line-height: 1.8;
                color: #444444;
            }
            .contact-info {
                text-align: center;
                padding: 20px;
                background-color: #f1f1f1;
                border-radius: 5px;
                margin-top: 20px;
            }
            .contact-info a {
                color: #0056b3;
                text-decoration: none;
            }
            .contact-info a:hover {
                text-decoration: underline;
            }
            footer {
                text-align: center;
                padding: 20px;
                font-size: 0.9em;
                color: #999999;
                margin-top: 20px;
                border-top: 1px solid #e0e0e0;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <header>
                <h1>Real Estate Insights by Michelle Sanchez</h1>
                <p>Welcome to today's update on the latest in real estate!</p>
                <p>Contact me anytime at <a href="mailto:Michelledsanchez413@gmail.com">Michelledsanchez413@gmail.com</a> or call me directly at 940-273-4848.</p>
                <p>Stay connected on <a href="https://www.facebook.com/MichelleSanchezRealtor">Facebook</a> for more updates and listings.</p>
            </header>
            <div class="news-container">
    """
    for news in news_items:
        html_content += f"""
            <div class="news-item">
                <h2>{news['title']}</h2>
                <img src="{news['image_url']}" alt="Image for {news['title']}">
                <p class="news-summary">{news['summary']}</p>
                <p><a href="{news['original_link']}" target="_blank">Read more</a></p>
            </div>
        """
    html_content += """
            </div>
            <div class="contact-info">
                <p>Contact me anytime at <a href="mailto:Michelledsanchez413@gmail.com">Michelledsanchez413@gmail.com</a> or call me directly at 940-273-4848.</p>
                <p>Stay connected on <a href="https://www.facebook.com/MichelleSanchezRealtor">Facebook</a> for more updates and listings.</p>
            </div>
        </div>
        <footer>
            <p>&copy; 2024 Michelle Sanchez Real Estate. All rights reserved.</p>
        </footer>
    </body>
    </html>
    """
    return html_content

# Function to process news and create images concurrently
def process_news_and_create_images(news_content):
    news_items = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_image = {
            executor.submit(create_image, f"An illustration representing the following real estate news: {news.strip()}"): news
            for news in news_content.split("\n\n")
        }
        for future in concurrent.futures.as_completed(future_to_image):
            news = future_to_image[future]
            try:
                image_url = future.result()
                news_items.append({
                    "title": f"Real Estate News {len(news_items) + 1}",
                    "summary": news.strip(),
                    "image_url": image_url,
                    "original_link": "https://example.com"  # Replace with actual link if available
                })
            except Exception as e:
                print(f"Error creating image for news: {news.strip()}. Error: {e}")

    return news_items

# Main workflow function
def main():
    try:
        # Fetch news content
        news_content = get_real_estate_news()

        # Process news and create images concurrently
        news_items = process_news_and_create_images(news_content)

        # Generate HTML content
        html_content = generate_html(news_items)

        # Save HTML content with UTF-8 encoding
        with open('real_estate_news_north_texasv4.html', 'w', encoding='utf-8') as f:
            f.write(html_content)
        print("HTML content has been saved to 'real_estate_news_north_texasv4.html'.")

        # Placeholder for Facebook posting
        print("Placeholder for Facebook posting")

    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()


Successfully retrieved news
Successfully created image for prompt: An illustration representing the following real estate news: 5. **Market Insights**: FOX 4 News Dallas-Fort Worth shares stories on the U.S. housing shortage, rising home insurance premiums, and the impact of mortgage rates on affordability. They also highlight local real estate investors' increased activity in purchasing homes and the challenges faced by homeowners in the region.
Successfully created image for prompt: An illustration representing the following real estate news: These updates provide a comprehensive view of the current real estate landscape in North Texas, encompassing both commercial and residential developments.
Successfully created image for prompt: An illustration representing the following real estate news: 1. **Data Center Launch**: Aligned has announced the construction of its largest data center in North Texas, a significant development in the region's tech sector.
Successfully created image for

In [6]:
import os
import requests
import json
import pandas as pd
import concurrent.futures
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# Set your API keys here
FALLBACK_PERPLEXITY_API_KEY = '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'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PERPLEXITY_API_KEY = os.getenv('PERPLEXITY_API_KEY', FALLBACK_PERPLEXITY_API_KEY)

# Check if the API keys are set
if not OPENAI_API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")
if not PERPLEXITY_API_KEY:
    raise ValueError("Please set the PERPLEXITY_API_KEY environment variable or provide a fallback API key.")

# Create a requests session with retries
session = requests.Session()
retry = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504])
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# Initialize the ChatOpenAI model
chat = ChatOpenAI(temperature=0.7, openai_api_key=OPENAI_API_KEY)

# 📊 Function to gather latest energy and financial services news
def get_energy_financial_news():
    query = "latest energy and financial services news"
    url = f"https://api.perplexity.ai/chat/completions"
    headers = {
        "Authorization": f"Bearer {PERPLEXITY_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "llama-3-sonar-small-32k-online",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": query}
        ],
        "max_tokens": 4000,
        "temperature": 0.7,
        "top_p": 0.95,
        "return_citations": True
    }
    response = session.post(url, headers=headers, data=json.dumps(payload))
    response.raise_for_status()
    print("Successfully retrieved news")
    return response.json()['choices'][0]['message']['content']

# 🖼️ Function to create an image for each news summary
def create_image(prompt):
    url = "https://api.openai.com/v1/images/generations"
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "model": "dall-e-3",
        "prompt": prompt,
        "n": 1,
        "size": "1024x1024"
    }
    response = session.post(url, headers=headers, json=data)
    response.raise_for_status()
    print(f"Successfully created image for prompt: {prompt}")
    return response.json()['data'][0]['url']

# 🖥️ Function to generate HTML content
def generate_html(news_items):
    html_content = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Energy & Financial Services by Trane Technologies</title>
        <style>
            body {
                font-family: 'Arial', sans-serif;
                background-color: #ffffff;
                color: #000000;
                margin: 0;
                padding: 0;
            }
            .container {
                max-width: 800px;
                margin: 40px auto;
                padding: 20px;
                background-color: #f7f7f7;
                border-radius: 5px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            }
            header {
                text-align: center;
                border-bottom: 2px solid #e0e0e0;
                padding-bottom: 20px;
                margin-bottom: 20px;
            }
            header h1 {
                font-size: 2.5em;
                margin: 0;
                font-weight: bold;
                color: #6400FF;
            }
            header p {
                font-size: 1.2em;
                color: #666666;
                margin: 10px 0 0;
            }
            .news-container {
                padding: 20px 0;
            }
            .news-item {
                border-bottom: 1px solid #e0e0e0;
                margin-bottom: 20px;
                padding-bottom: 20px;
            }
            .news-item:last-child {
                border-bottom: none;
                margin-bottom: 0;
                padding-bottom: 0;
            }
            .news-item img {
                width: 100%;
                height: auto;
                border-radius: 5px;
                margin-bottom: 10px;
            }
            .news-summary {
                font-size: 1.1em;
                line-height: 1.8;
                color: #444444;
            }
            .contact-info {
                text-align: center;
                padding: 20px;
                background-color: #f1f1f1;
                border-radius: 5px;
                margin-top: 20px;
            }
            .contact-info a {
                color: #6400FF;
                text-decoration: none;
            }
            .contact-info a:hover {
                text-decoration: underline;
            }
            footer {
                text-align: center;
                padding: 20px;
                font-size: 0.9em;
                color: #999999;
                margin-top: 20px;
                border-top: 1px solid #e0e0e0;
            }
        </style>
    </head>
    <body>
        <div class="container">
            <header>
                <h1>Energy & Financial Services by Trane Technologies</h1>
                <p>Welcome to the latest updates in energy and financial services!</p>
                <p>Contact us anytime at <a href="mailto:support@tranetechnologies.com">support@tranetechnologies.com</a> or call us directly at 1-800-555-1234.</p>
                <p>Stay connected on <a href="https://www.facebook.com/TraneTechnologies">Facebook</a> for more updates and insights.</p>
            </header>
            <div class="news-container">
    """
    for news in news_items:
        html_content += f"""
            <div class="news-item">
                <h2>{news['title']}</h2>
                <img src="{news['image_url']}" alt="Image for {news['title']}">
                <p class="news-summary">{news['summary']}</p>
                <p><a href="{news['original_link']}" target="_blank">Read more</a></p>
            </div>
        """
    html_content += """
            </div>
            <div class="contact-info">
                <p>Contact us anytime at <a href="mailto:support@tranetechnologies.com">support@tranetechnologies.com</a> or call us directly at 1-800-555-1234.</p>
                <p>Stay connected on <a href="https://www.facebook.com/TraneTechnologies">Facebook</a> for more updates and insights.</p>
            </div>
        </div>
        <footer>
            <p>&copy; 2024 Trane Technologies. All rights reserved.</p>
        </footer>
    </body>
    </html>
    """
    return html_content

# Function to process news and create images concurrently
def process_news_and_create_images(news_content):
    news_items = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_image = {
            executor.submit(create_image, f"An illustration representing the following news: {news.strip()}"): news
            for news in news_content.split("\n\n")
        }
        for future in concurrent.futures.as_completed(future_to_image):
            news = future_to_image[future]
            try:
                image_url = future.result()
                news_items.append({
                    "title": f"News {len(news_items) + 1}",
                    "summary": news.strip(),
                    "image_url": image_url,
                    "original_link": "https://example.com"  # Replace with actual link if available
                })
            except Exception as e:
                print(f"Error creating image for news: {news.strip()}. Error: {e}")

    return news_items

# Main workflow function
def main():
    try:
        # Fetch news content
        news_content = get_energy_financial_news()

        # Process news and create images concurrently
        news_items = process_news_and_create_images(news_content)

        # Generate HTML content
        html_content = generate_html(news_items)

        # Save HTML content with UTF-8 encoding
        with open('energy_financial_news_trane.html', 'w', encoding='utf-8') as f:
            f.write(html_content)
        print("HTML content has been saved to 'energy_financial_news_trane.html'.")

        # Placeholder for Facebook posting
        print("Placeholder for Facebook posting")

    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()


Successfully retrieved news
Error creating image for news: 1. **Market Updates**: The Dow Jones Industrial Average closed at 39,411.21, marking a 0.67% increase. The S&P 500 saw a 0.31% decline, while the NASDAQ experienced a 1.09% loss.. Error: 429 Client Error: Too Many Requests for url: https://api.openai.com/v1/images/generations
Error creating image for news: 3. **OPEC's Influence**: The International Energy Agency (IEA) suggested that OPEC's influence on global oil prices might diminish in the near future.. Error: 429 Client Error: Too Many Requests for url: https://api.openai.com/v1/images/generations
Error creating image for news: 4. **Chevron Australia Strikes**: Chevron Australia employees are set to resume strikes next week due to stalled negotiations.. Error: 429 Client Error: Too Many Requests for url: https://api.openai.com/v1/images/generations
Error creating image for news: 5. **Russia's Diesel Exports**: Russia eased its diesel export restrictions, leading to lower pri